<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Using a Local LLM</h1>

Below is an example of using a local LLM to perform evals. In this example we will be using [ollama](https://ollama.com/)

In [1]:
!pip install -qq "arize-phoenix" "litellm" openinference-instrumentation-litellm

In [2]:
import nest_asyncio

nest_asyncio.apply()

Make sure you have ollama installed and running your model of choice. In this example we'll be using `ollama/llama3.2:1b`.

```bash
ollama run ollama/llama3.2:1b
```
We'll also need to set the `OLLAMA_API_BASE` environment variable to point to your local ollama instance. The port `11434` is the default port for ollama.

In [3]:
import os

from phoenix.evals import LiteLLMModel

os.environ["OLLAMA_API_BASE"] = "http://localhost:11434"

model = LiteLLMModel(model="ollama/llama3.2:1b")

Optional: Instrument the model to trace requests and responses. You can still use the model, and Phoenix evaluations, without this next block of code, however this will allow you to trace the requests made to your local LLM in Phoenix's UI. Phoenix can be run locally in this notebook, [locally on your machine](https://docs.arize.com/phoenix/deployment/environments), or [accessed in the cloud](https://docs.arize.com/phoenix/quickstart). In keeping with the spirit of this tutorial, we'll run Phoenix locally in the notebook.

In [ ]:
from openinference.instrumentation.litellm import LiteLLMInstrumentor

import phoenix as px
from phoenix.otel import register

px.launch_app()  # remove this line and run `pip install arize-phoenix` + `phoenix serve` in a terminal window to run Phoenix locally on your machine

tp = register()
LiteLLMInstrumentor().instrument(tracer_provider=tp)

In [ ]:
model("Hello, world!")

In [ ]:
from phoenix.evals import download_benchmark_dataset

df = download_benchmark_dataset(
    task="binary-relevance-classification", dataset_name="wiki_qa-train"
)
df.head()

In [ ]:
from phoenix.evals import (
    RAG_RELEVANCY_PROMPT_RAILS_MAP,
    RAG_RELEVANCY_PROMPT_TEMPLATE,
    llm_classify,
)

N_EVAL_SAMPLE_SIZE = 100

df_sample = df.sample(n=N_EVAL_SAMPLE_SIZE)

df_sample = df_sample.rename(
    columns={
        "query_text": "input",
        "document_text": "reference",
    },
)

rails = list(RAG_RELEVANCY_PROMPT_RAILS_MAP.values())

relevance_df = llm_classify(
    dataframe=df_sample,
    template=RAG_RELEVANCY_PROMPT_TEMPLATE,
    model=model,
    rails=rails,
    concurrency=20,
)

In [ ]:
relevance_df.head(20)